# GEEを使ったサンプル

MODISデータを触ってみる

参考：https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api

In [2]:
!pip install earthengine-api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.0/242.0 KB 6.3 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.2/829.2 KB 9.8 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 KB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.2/167.2 KB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 KB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 KB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 KB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.7/211.7 KB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 403.

In [3]:
import ee

In [4]:
ee.Authenticate()

Enter verification code:  4/1AdQt8qhA0G9J1vDI59PLqCYKNN57OzBz7CXi6UuGMXtG__qAmQkntSYQf_w



Successfully saved authorization token.


In [5]:
# The initialization step verifies that valid credentials have been created and populates the Python client library with methods that the backend server supports.
ee.Initialize()

# GEE data type
In the Earth Engine Data Catalog, datasets can be of different types:

- Features which are geometric objects with a list of properties. For example, a watershed with some properties such as name and area, is an `ee.Feature`.

- Images which are like features, but may include several bands. For example, the ground elevation given by the USGS here is `an ee.Image`.

- Collections which are groups of features or images. For example, the Global Administrative Unit Layers giving administrative boundaries is a `ee.FeatureCollection` and the MODIS Land Surface Temperature dataset is an `ee.ImageCollection`.

The dataset descriptions provide us with all the information we need to import and manipulate these datasets:

- the availability
- the provider
- the Earth Engine Snippet
- the available bands associated with images in the collection

更なる詳細は：https://developers.google.com/earth-engine


利用可能なデータカタログは　https://developers.google.com/earth-engine/datasets/catalog/　で検索できる。各データセットの詳細ページにデータ取得のコマンドサンプルがあるためコピペするだけ。

APIリファレンスは https://developers.google.com/earth-engine/api_docs

In [11]:
# 見たい範囲を設定
# Define the urban location of interest as a point near Lyon, France.
u_lon = 4.8148
u_lat = 45.7758
u_poi = ee.Geometry.Point(u_lon, u_lat)

# Define the rural location of interest as a point away from the city.
r_lon = 5.175964
r_lat = 45.574064
r_poi = ee.Geometry.Point(r_lon, r_lat)

データの読み込み

In [15]:
# Import the MODIS land cover collection.
lc = ee.ImageCollection('MODIS/006/MCD12Q1')

In [16]:
# Import the MODIS land surface temperature collection.
lst = ee.ImageCollection('MODIS/006/MOD11A1')

In [ ]:
# Import the USGS ground elevation image.
elv = ee.Image('USGS/SRTMGL1_003')

## LST(Land Surface Temperature)

In [18]:
# 日付によるフィルタリング
# Initial date of interest (inclusive).
i_date = '2017-01-01'
# Final date of interest (exclusive).
f_date = '2020-01-01'
# Selection of appropriate bands and dates for LST.
lst = lst.select('LST_Day_1km', 'QC_Day').filterDate(i_date, f_date)

In [19]:
# Calculate and print the mean value of the LST collection at the point.
lst_urban_point = lst.mean().sample(u_poi, scale).first().get('LST_Day_1km').getInfo()
print('Average daytime LST at urban point:', round(lst_urban_point*0.02 -273.15, 2), '°C')

Average daytime LST at urban point: 22.06 °C


In [22]:
# Define a region of interest with a buffer zone of 1000 km around Lyon.
roi = u_poi.buffer(1e6)
roi

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "Geometry.buffer",
    "arguments": {
      "distance": {
        "constantValue": 1000000.0
      },
      "geometry": {
        "functionInvocationValue": {
          "functionName": "GeometryConstructors.Point",
          "arguments": {
            "coordinates": {
              "constantValue": [
                4.8148,
                45.7758
              ]
            }
          }
        }
      }
    }
  }
})

In [23]:
# Reduce the LST collection by mean.
lst_img = lst.mean()

# Adjust for scale factor.
lst_img = lst_img.select('LST_Day_1km').multiply(0.02)

# Convert Kelvin to Celsius.
lst_img = lst_img.select('LST_Day_1km').add(-273.15)

In [25]:
# Create a URL to the styled image for a region around France.
url = lst_img.getThumbUrl({
    'min': 10, 'max': 30, 'dimensions': 512, 'region': roi,
    'palette': ['blue', 'yellow', 'orange', 'red']})

In [26]:
url

'https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/c2f5dfbce16846626c79f7ce96e533c5-773bcfc756abbae82b527a50aa778665:getPixels'

In [28]:
# Display the thumbnail land surface temperature in France.
from IPython.display import Image
Image(url=url)

## ELV(Elevation)

In [14]:
scale = 1000 # scale in meters
elv.sample(u_poi, scale).first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '0',
 'properties': {'elevation': 196}}

In [31]:
# Make pixels with elevation below sea level transparent.
elv_img = elv.updateMask(elv.gt(0))

# Display the thumbnail of styled elevation in France.
Image(url=elv_img.getThumbURL({
    'min': 0, 'max': 2000, 'dimensions': 512, 'region': roi,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}))

## LC(Land Cover)

In [ ]:
# Print the land cover type at the point.
lc_urban_point = lc.first().sample(u_poi, scale).first().get('LC_Type1').getInfo()
print('Land cover value at urban point is:', lc_urban_point)

Land cover value at urban point is: 13


# GeoTIFFファイルで出力